# GPT-2 실습: Decision Transformer를 위한 기초

이 노트북은 GPT-2의 핵심 구조를 PyTorch로 직접 구현하며 배웁니다.

## 학습 목표

1. Self-Attention 메커니즘 이해
2. Transformer Block 구조 파악
3. Causal Masking의 중요성 이해
4. 간단한 텍스트 생성 경험

---

In [ ]:
# ============================================================
# 📦 패키지 임포트
# ============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

print(f"🐍 PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print()
print("==================================================")
준비 완료!")


## 1. GPT-2 개요

### 1.1 전체 아키텍처

```mermaid
flowchart TB
    subgraph Input["📥 입력"]
        Tokens["Token IDs<br/>(B, T)"]
    end

    subgraph Embedding["1️⃣ 임베딩"]
        TE["Token Embedding<br/>(vocab_size → n_embd)"]
        PE["Positional Embedding<br/>(block_size → n_embd)"]
        Add["임베딩 합산"]
        Drop["Dropout"]
    end

    subgraph Blocks["2️⃣ Transformer Blocks × N"]
        direction TB
        B1["Block 1"]
        B2["Block 2"]
        B3["..."]
        B4["Block N"]
    end

    subgraph Head["3️⃣ 출력 헤드"]
        LN["LayerNorm"]
        Linear["Linear<br/>(n_embd → vocab_size)"]
    end

    subgraph Output["📤 출력"]
        Logits["Logits<br/>(B, T, vocab_size)"]
        NextToken["Next Token"]
    end

    Tokens --> TE
    Tokens --> PE
    TE --> Add
    PE --> Add
    Add --> Drop --> Blocks --> LN --> Linear --> Logits --> NextToken

    style Blocks fill:#e3f2fd
    style NextToken fill:#c8e6c9
    style Add fill:#fff9c4
```

### 1.2 GPT-2는 무엇인가?

**GPT-2** = **G**enerative **P**re-trained **T**ransformer-2

```mermaid
mindmap
    root((GPT-2))
        Generative["Generative<br/>다음 토큰 생성"]
        Pretrained["Pre-trained<br/>대규모 텍스트 데이터"]
        Transformer["Transformer<br/>Self-attention 기반"]
        Autoregressive["Autoregressive<br/>이전 토큰만 참조"]
```

- **Generative**: 다음 토큰을 생성하는 모델
- **Pre-trained**: 대규모 텍스트 데이터로 사전 학습
- **Transformer**: Self-attention 기반 아키텍처
- **Autoregressive**: Causal masking으로 이전 토큰만 참조

### 1.3 동작 예시

```
입력: "The cat sat on the"
출력: "mat" (가장 높은 확률)

확률 분포:
P(mat | The, cat, sat, on, the)    = 0.23  ← 선택
P(floor | The, cat, sat, on, the)  = 0.15
P(bed | The, cat, sat, on, the)    = 0.10
P(couch | The, cat, sat, on, the)  = 0.08
...
```

## 2. Self-Attention 구현

### 2.1 Attention의 직관적 이해

```mermaid
flowchart LR
    subgraph Question["Query: 'cat'이 무엇을 집중해서 보는가?"]
        Q["cat (Query)"]
    end

    subgraph Keys["Keys (문맥 단어들)"]
        K1["The"]
        K2["cat"]
        K3["sat"]
        K4["on"]
        K5["the"]
        K6["mat"]
    end

    subgraph Attention["Attention Weights"]
        A1["0.1"]
        A2["0.5 ⭐"]
        A3["0.3"]
        A4["0.05"]
        A5["0.05"]
        A6["0.0"]
    end

    Q --> A1
    Q --> A2
    Q --> A3
    Q --> A4
    Q --> A5
    Q --> A6

    K1 -.-> A1
    K2 -.-> A2
    K3 -.-> A3
    K4 -.-> A4
    K5 -.-> A5
    K6 -.-> A6

    style Q fill:#ffcdd2
    style A2 fill:#c8e6c9
    style K2 fill:#c8e6c9
```

**핵심 아이디어**: Attention = 가중치 합 (Weighted Sum)
- Query: "내가 무엇을 찾는가?"
- Key: "각 토큰이 무엇을 제공하는가?"
- Value: "실제 내용是什么?"

### 2.2 Scaled Dot-Product Attention

```mermaid
flowchart TB
    subgraph Input["입력"]
        Q["Query Matrix<br/>Q (B, n_head, T, d_k)"]
        K["Key Matrix<br/>K (B, n_head, T, d_k)"]
        V["Value Matrix<br/>V (B, n_head, T, d_k)"]
    end

    subgraph Step1["Step 1: Similarity"]
        MatMul["Q @ K^T<br/>(B, n_head, T, T)"]
    end

    subgraph Step2["Step 2: Scale"]
        Scale["/ √d_k<br/>스케일링"]
    end

    subgraph Step3["Step 3: Mask (Optional)"]
        Mask["Causal Mask<br/>하삼각 마스크"]
    end

    subgraph Step4["Step 4: Normalize"]
        Softmax["Softmax<br/>확률 분포"]
    end

    subgraph Step5["Step 5: Weighted Sum"]
        AttnOut["@ V<br/>(B, n_head, T, d_k)"]
    end

    Q --> MatMul
    K --> MatMul
    MatMul --> Scale --> Mask --> Softmax --> AttnOut
    V --> AttnOut

    style MatMul fill:#ffcdd2
    style Softmax fill:#c8e6c9
    style AttnOut fill:#fff9c4
```

**공식**: 
```
Attention(Q, K, V) = softmax(Q @ K^T / √d_k) @ V
```

### 2.3 왜 Multi-Head인가?

```mermaid
flowchart TB
    subgraph Single["Single Head"]
        S1["하나의 Attention<br/>모든 정보를 한 곳에"]
        SOut["출력"]
    end

    subgraph Multi["Multi-Head"]
        M1["Head 1: 문법 정보"]
        M2["Head 2: 의미 정보"]
        M3["Head 3: 위치 정보"]
        M4["Head 4: ..."]
        MOut["결합: 다양한 관점"]
    end

    subgraph Compare["비교"]
        C1["Single: 하나의 표현 공간"]
        C2["Multi: 여러 표현 공간<br/>더 풍부한 표현"]
    end

    S1 --> SOut
    M1 --> MOut
    M2 --> MOut
    M3 --> MOut
    M4 --> MOut

    SOut --> C1
    MOut --> C2

    style Single fill:#ffcdd2
    style Multi fill:#c8e6c9
```

**장점**:
- **다양한 표현**: 각 Head가 다른 관점에서 정보 포착
- **병렬 처리**: 동시에 여러 Attention 계산
- **표현력 향상**: 더 복잡한 패턴 학습 가능

### 2.4 Multi-Head Attention 구조

```mermaid
flowchart TB
    subgraph Input["입력: x (B, T, C)"]
        X["x"]
    end

    subgraph Projections["Q, K, V Projections"]
        Q["Linear → Q<br/>(B, T, C)"]
        K["Linear → K<br/>(B, T, C)"]
        V["Linear → V<br/>(B, T, C)"]
    end

    subgraph Split["Multi-Head 분할"]
        S1["Q → (B, n_head, T, head_dim)"]
        S2["K → (B, n_head, T, head_dim)"]
        S3["V → (B, n_head, T, head_dim)"]
        Note1["head_dim = C / n_head"]
    end

    subgraph Attn["Attention 계산"]
        MatMul["Q @ K^T<br/>(B, n_head, T, T)"]
        Scale["/ √d_k"]
        Mask["Causal Mask"]
        Softmax["Softmax"]
        AttnOut["@ V<br/>(B, n_head, T, head_dim)"]
    end

    subgraph Concat["Multi-Head 결합"]
        Concat["Transpose + Reshape<br/>(B, n_head, T, head_dim)<br/>→ (B, T, C)"]
    end

    subgraph Output["출력 Projection"]
        Proj["Linear(C → C)"]
        Dropout["Dropout"]
    end

    X --> Q
    X --> K
    X --> V

    Q --> S1
    K --> S2
    V --> S3

    S1 --> MatMul
    S2 --> MatMul
    S3 --> AttnOut

    MatMul --> Scale --> Mask --> Softmax --> AttnOut
    AttnOut --> Concat --> Proj --> Dropout

    style MatMul fill:#ffcdd2
    style AttnOut fill:#c8e6c9
    style Softmax fill:#fff9c4
    style Concat fill:#e3f2fd
```

**Shape 변화 예시** (n_head=4, C=128):
- 입력: `(B, T, 128)`
- Q, K, V: `(B, T, 128)`
- Split 후: `(B, 4, T, 32)` ← head_dim = 128/4 = 32
- Attention 후: `(B, 4, T, 32)`
- Concat 후: `(B, T, 128)`

### 2.5 Multi-Head Attention의 효과

```mermaid
flowchart LR
    subgraph Sentence["문장: 'The cat sat on the mat'"]
        W1["cat"]
        W2["mat"]
    end

    subgraph Heads["각 Head의 역할"]
        H1["Head 1:<br/>문법 관계<br/>cat ↔ sat"]
        H2["Head 2:<br/>대명사 참조<br/>The ↔ cat"]
        H3["Head 3:<br/>전치사 관계<br/>on ↔ mat"]
        H4["Head 4:<br/>의미 관계<br/>cat ↔ mat"]
    end

    subgraph Result["결과"]
        R1["통합된 표현:<br/>문법 + 의미 + 구조"]
    end

    W1 --> H1
    W1 --> H2
    W1 --> H4
    W2 --> H3
    W2 --> H4

    H1 --> R1
    H2 --> R1
    H3 --> R1
    H4 --> R1

    style Heads fill:#e3f2fd
    style Result fill:#c8e6c9
```

### 2.6 Causal Masking 필요性

In [ ]:
# ============================================================
# 🧠 Attention 메커니즘 시각화
# ============================================================

import matplotlib.pyplot as plt
import seaborn as sns

# 예시 문장: "The cat sat on the mat"
tokens = ["The", "cat", "sat", "on", "the", "mat"]

# 가짜 attention weights (실제로는 모델이 학습)
# 각 행은 해당 단어가 다른 단어에 얼마나 attention하는지
attention_weights = torch.tensor([
    [0.6, 0.3, 0.1, 0.0, 0.0, 0.0],  # "The"는 "cat"에 많이 attend
    [0.2, 0.5, 0.2, 0.1, 0.0, 0.0],  # "cat"은 본인과 "sat"에 attend
    [0.1, 0.3, 0.4, 0.1, 0.1, 0.0],  # "sat"은 주변에 attend
    [0.0, 0.1, 0.2, 0.4, 0.2, 0.1],  # "on"은 주변에 attend
    [0.0, 0.0, 0.1, 0.2, 0.5, 0.2],  # "the"는 "mat"에 attend
    [0.0, 0.0, 0.1, 0.1, 0.3, 0.5],  # "mat"은 본인과 이전 단어에 attend
])

# Causal masking 적용 (미래 단어는 볼 수 없음)
causal_mask = torch.tril(torch.ones(6, 6))
attention_weights = attention_weights * causal_mask
attention_weights = attention_weights / attention_weights.sum(dim=1, keepdim=True)

# 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights.numpy(), 
            xticklabels=tokens, 
            yticklabels=tokens,
            cmap='Blues',
            annot=True,
            fmt='.2f')
plt.title('Attention Pattern (Causal Masking Applied)')
plt.xlabel('Key (attend to)')
plt.ylabel('Query (current token)')
plt.tight_layout()
plt.show()

print("📊 Attention Pattern 해석:")
print("  - 각 행은 해당 단어가 문맥에서 어떤 단어에 집중하는지")
print("  - Causal masking으로 미래 단어는 0 (볼 수 없음)")

### 2.2 Self-Attention 구현

In [ ]:
# ============================================================
# 🔧 Causal Self-Attention 구현
# ============================================================

class CausalSelfAttention(nn.Module):
    """
    GPT-2의 Causal Self-Attention
    
    핵심: 각 토큰이 자신과 이전 토큰만 볼 수 있음
    """
    def __init__(self, n_embd, n_head, block_size, attn_pdrop=0.1, resid_pdrop=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        
        self.n_head = n_head
        self.n_embd = n_embd
        
        # Q, K, V projections
        self.key = nn.Linear(n_embd, n_embd)
        self.query = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)
        
        # Dropout
        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)
        
        # Output projection
        self.proj = nn.Linear(n_embd, n_embd)
        
        # Causal mask (하삼각 행렬)
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(block_size, block_size))
                 .view(1, 1, block_size, block_size)
        )
    
    def forward(self, x):
        """
        Args:
            x: (B, T, C) - Batch, Time, Channels
        Returns:
            y: (B, T, C)
        """
        B, T, C = x.size()
        
        # Q, K, V 계산
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        v = self.value(x) # (B, T, C)
        
        # Multi-head로 분할
        head_dim = C // self.n_head
        k = k.view(B, T, self.n_head, head_dim).transpose(1, 2)  # (B, n_head, T, head_dim)
        q = q.view(B, T, self.n_head, head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_head, head_dim).transpose(1, 2)
        
        # Attention scores
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        
        # Causal masking
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        
        # Attention output
        y = att @ v  # (B, n_head, T, head_dim)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Output projection
        y = self.resid_drop(self.proj(y))
        
        return y

# 테스트
B, T, C = 2, 10, 32
attn = CausalSelfAttention(n_embd=C, n_head=4, block_size=128)
x = torch.randn(B, T, C)
y = attn(x)

print(f"✅ Attention 테스트 통과!")
print(f"   입력 shape: {x.shape}")
print(f"   출력 shape: {y.shape}")
print(f"   Head 수: {attn.n_head}")
print(f"   Head 차원: {C // attn.n_head}")

# ============================================================
# 🔍 Multi-Head Attention 내부 들여다보기
# ============================================================

def inspect_multihead_attention():
    """Multi-Head Attention의 내부 동작을 자세히 확인"""
    
    # 설정
    B, T, C = 1, 5, 12  # 배치 1, 시퀀스 5, 차원 12
    n_head = 3
    head_dim = C // n_head
    
    # 모델 생성
    attn_layer = CausalSelfAttention(n_embd=C, n_head=n_head, block_size=128)
    
    # 입력
    x = torch.randn(B, T, C)
    
    # 각 단계별로 확인
    with torch.no_grad():
        # Q, K, V 계산
        k = attn_layer.key(x)   # (B, T, C)
        q = attn_layer.query(x) # (B, T, C)
        v = attn_layer.value(x) # (B, T, C)
        
        print("📊 Q, K, V Projection:")
        print(f"   Q shape: {q.shape}")  # (1, 5, 12)
        print(f"   K shape: {k.shape}")
        print(f"   V shape: {v.shape}")
        
        # Multi-head 분할
        k = k.view(B, T, n_head, head_dim).transpose(1, 2)  # (B, n_head, T, head_dim)
        q = q.view(B, T, n_head, head_dim).transpose(1, 2)
        v = v.view(B, T, n_head, head_dim).transpose(1, 2)
        
        print(f"\n📊 Multi-Head 분할 후:")
        print(f"   Q shape: {q.shape}")  # (1, 3, 5, 4) - 3개 head, 각 head_dim=4
        print(f"   K shape: {k.shape}")
        print(f"   V shape: {v.shape}")
        print(f"   각 Head 차원: {head_dim}")
        
        # Attention scores (각 head별)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        print(f"\n📊 Attention Scores (before masking): {att.shape}")  # (1, 3, 5, 5)
        print(f"   각 Head가 각 토큰 쌍의 similarity 계산")
        
        # Causal masking
        mask = attn_layer.mask[:, :, :T, :T]
        att_masked = att.masked_fill(mask == 0, float('-inf'))
        att_probs = F.softmax(att_masked, dim=-1)
        
        print(f"\n📊 Attention Probabilities (after masking & softmax):")
        for h in range(n_head):
            print(f"\n   Head {h+1}의 Attention 패턴:")
            for t in range(T):
                probs = att_probs[0, h, t, :t+1].numpy()  # 현재 토큰까지
                print(f"      토큰 {t} → {[f'{p:.2f}' for p in probs]}")
        
        # Weighted sum
        y = att_probs @ v  # (1, 3, 5, 4)
        print(f"\n📊 Weighted Sum (각 Head의 출력): {y.shape}")
        
        # Head 결합
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        print(f"📊 Head 결합 후: {y.shape}")
        
        # Output projection
        y_final = attn_layer.proj(attn_layer.resid_drop(y))
        print(f"📊 최종 출력: {y_final.shape}")

print("\n" + "="*60)
print("🔍 Multi-Head Attention 내부 분석")
print("="*60)
inspect_multihead_attention()

In [ ]:
# ============================================================
# 🔧 Causal Self-Attention 구현
# ============================================================

class CausalSelfAttention(nn.Module):
    """
    GPT-2의 Causal Self-Attention
    
    핵심: 각 토큰이 자신과 이전 토큰만 볼 수 있음
    """
    def __init__(self, n_embd, n_head, block_size, attn_pdrop=0.1, resid_pdrop=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        
        self.n_head = n_head
        self.n_embd = n_embd
        
        # Q, K, V projections
        self.key = nn.Linear(n_embd, n_embd)
        self.query = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)
        
        # Dropout
        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)
        
        # Output projection
        self.proj = nn.Linear(n_embd, n_embd)
        
        # Causal mask (하삼각 행렬)
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(block_size, block_size))
                 .view(1, 1, block_size, block_size)
        )
    
    def forward(self, x):
        """
        Args:
            x: (B, T, C) - Batch, Time, Channels
        Returns:
            y: (B, T, C)
        """
        B, T, C = x.size()
        
        # Q, K, V 계산
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        v = self.value(x) # (B, T, C)
        
        # Multi-head로 분할
        head_dim = C // self.n_head
        k = k.view(B, T, self.n_head, head_dim).transpose(1, 2)  # (B, n_head, T, head_dim)
        q = q.view(B, T, self.n_head, head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_head, head_dim).transpose(1, 2)
        
        # Attention scores
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        
        # Causal masking
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        
        # Attention output
        y = att @ v  # (B, n_head, T, head_dim)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Output projection
        y = self.resid_drop(self.proj(y))
        
        return y

# 테스트
B, T, C = 2, 10, 32
attn = CausalSelfAttention(n_embd=C, n_head=4, block_size=128)
x = torch.randn(B, T, C)
y = attn(x)

print(f"✅ Attention 테스트 통과!")
print(f"   입력 shape: {x.shape}")
print(f"   출력 shape: {y.shape}")

## 3. Transformer Block

### 3.1 Block 구조

```mermaid
flowchart TB
    subgraph Block["Transformer Block"]
        direction TB
        
        subgraph Layer1["Layer 1: Attention"]
            LN1["LayerNorm(x)"]
            Attn["Self-Attention(x)"]
            Res1["x + Attn(LN1(x))<br/>Residual Connection"]
        end
        
        subgraph Layer2["Layer 2: MLP"]
            LN2["LayerNorm(x)"]
            MLP["MLP(x)<br/>Linear → GELU → Linear"]
            Res2["x + MLP(LN2(x))<br/>Residual Connection"]
        end
    end

    Input["x (B, T, C)"] --> LN1
    LN1 --> Attn --> Res1 --> LN2 --> MLP --> Res2 --> Output["y (B, T, C)"]

    style Attn fill:#ffcdd2
    style MLP fill:#c8e6c9
    style Res1 fill:#fff9c4
    style Res2 fill:#fff9c4
```

### 3.2 Pre-LN vs Post-LN

```mermaid
flowchart LR
    subgraph PreLN["Pre-LN (GPT-2 스타일)"]
        direction TB
        P1["x"]
        P2["LN(x)"]
        P3["SubLayer(LN(x))"]
        P4["x + SubLayer(LN(x))"]
        
        P1 --> P2 --> P3 --> P4
    end

    subgraph PostLN["Post-LN (원본 Transformer)"]
        direction TB
        L1["x"]
        L2["SubLayer(x)"]
        L3["LN(x + SubLayer(x))"]
        
        L1 --> L2 --> L3
    end

    style PreLN fill:#c8e6c9
    style PostLN fill:#ffcdd2
```

**GPT-2는 Pre-LN 사용** → 학습 안정성 향상

### 3.3 MLP (Feed Forward Network)

```mermaid
flowchart LR
    subgraph MLP["Feed Forward Network"]
        In["x (B, T, C)"]
        
        L1["Linear<br/>C → 4×C"]
        Act["GELU Activation"]
        L2["Linear<br/>4×C → C"]
        Drop["Dropout"]
        
        Out["y (B, T, C)"]
    end

    In --> L1 --> Act --> L2 --> Drop --> Out

    style L1 fill:#e3f2fd
    style L2 fill:#e3f2fd
    style Act fill:#fff9c4
```

In [ ]:
# ============================================================
# 🧱 Transformer Block 구현
# ============================================================

class TransformerBlock(nn.Module):
    """
    Transformer Block: Attention + MLP
    """
    def __init__(self, n_embd, n_head, block_size, resid_pdrop=0.1, attn_pdrop=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = CausalSelfAttention(n_embd, n_head, block_size, attn_pdrop, resid_pdrop)
        
        # MLP: 4배 확장 후 축소
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(resid_pdrop),
        )
    
    def forward(self, x):
        # Pre-LN + Residual (GPT-2 스타일)
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

# 테스트
block = TransformerBlock(n_embd=32, n_head=4, block_size=128)
x = torch.randn(2, 10, 32)
y = block(x)

print(f"✅ Transformer Block 테스트 통과!")
print(f"   출력 shape: {y.shape}")

## 4. GPT-2 모델

In [ ]:
# ============================================================
# 🤖 GPT-2 모델 구현
# ============================================================

class GPT(nn.Module):
    """
    간단한 GPT-2 모델
    """
    def __init__(self, vocab_size, block_size, n_layer=6, n_head=8, n_embd=512):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        
        # Token & Positional embeddings
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.drop = nn.Dropout(0.1)
        
        # Transformer blocks
        self.blocks = nn.Sequential(*[
            TransformerBlock(n_embd, n_head, block_size)
            for _ in range(n_layer)
        ])
        
        # Final layer norm and head
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)
    
    def forward(self, idx, targets=None):
        """
        Args:
            idx: (B, T) - Token IDs
            targets: (B, T) - Target tokens (optional)
        Returns:
            logits: (B, T, vocab_size)
            loss: scalar (optional)
        """
        B, T = idx.size()
        assert T <= self.block_size
        
        # Embeddings
        tok_emb = self.tok_emb(idx)  # (B, T, n_embd)
        pos_emb = self.pos_emb[:, :T, :]  # (1, T, n_embd)
        x = self.drop(tok_emb + pos_emb)
        
        # Transformer blocks
        x = self.blocks(x)
        
        # Final layer norm and head
        x = self.ln_f(x)
        logits = self.head(x)  # (B, T, vocab_size)
        
        # Loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=-1
            )
        
        return logits, loss

# 테스트
model = GPT(vocab_size=100, block_size=32, n_layer=2, n_head=2, n_embd=64)
idx = torch.randint(0, 100, (2, 10))
logits, loss = model(idx)

print(f"✅ GPT 모델 테스트 통과!")
print(f"   출력 shape: {logits.shape}")
print(f"   파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

## 5. 간단한 텍스트 생성

### 5.1 문자 단위 토크나이저

In [ ]:
# ============================================================
# 📝 문자 단위 토크나이저
# ============================================================

class CharTokenizer:
    """간단한 문자 단위 토크나이저"""
    def __init__(self, text):
        self.chars = sorted(list(set(text)))
        self.char2idx = {ch: i for i, ch in enumerate(self.chars)}
        self.idx2char = {i: ch for i, ch in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
    
    def encode(self, text):
        return torch.tensor([self.char2idx[ch] for ch in text])
    
    def decode(self, tokens):
        if isinstance(tokens, torch.Tensor):
            tokens = tokens.tolist()
        return ''.join([self.idx2char[idx] for idx in tokens])

# 테스트
text = "hello world"
tokenizer = CharTokenizer(text)

print(f"📝 어휘 크기: {tokenizer.vocab_size}")
print(f"📝 문자: {tokenizer.chars}")
print(f"📝 'hello' 인코딩: {tokenizer.encode('hello')}")
print(f"📝 디코딩: '{tokenizer.decode(tokenizer.encode('hello'))}'")

### 5.2 텍스트 생성 함수

In [ ]:
# ============================================================
# 🎲 텍스트 생성 함수
# ============================================================

def generate(model, tokenizer, prompt, max_tokens=100, temperature=1.0):
    """
    텍스트 생성
    
    Args:
        model: GPT 모델
        tokenizer: 토크나이저
        prompt: 시작 텍스트
        max_tokens: 생성할 최대 토큰 수
        temperature: 샘플링 온도 (낮을수록 보수적)
    """
    model.eval()
    tokens = tokenizer.encode(prompt).unsqueeze(0)  # (1, T)
    
    with torch.no_grad():
        for _ in range(max_tokens):
            # 다음 토큰 예측
            logits, _ = model(tokens)
            logits = logits[0, -1, :] / temperature  # (vocab_size,)
            
            # 샘플링
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # 토큰 추가
            tokens = torch.cat([tokens, next_token.unsqueeze(0)], dim=1)
            
            # EOS 체크 (선택사항)
            if next_token.item() == tokenizer.char2idx.get('.', -1):
                break
    
    return tokenizer.decode(tokens[0])

# 테스트를 위한 간단한 훈련
def simple_training_example():
    """간단한 훈련 예제"""
    # 데이터
    text = "The cat sat on the mat. The dog sat on the log. " * 100
    tokenizer = CharTokenizer(text)
    
    # 모델
    model = GPT(
        vocab_size=tokenizer.vocab_size,
        block_size=64,
        n_layer=2,
        n_head=2,
        n_embd=64
    )
    
    # 옵티마이저
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    
    # 훈련
    tokens = tokenizer.encode(text)
    print("\n🎯 훈련 시작...")
    
    for epoch in range(50):
        # 랜덤 배치 샘플링
        start_idx = torch.randint(0, len(tokens) - 64, (1,)).item()
        x = tokens[start_idx:start_idx + 64].unsqueeze(0)
        y = tokens[start_idx + 1:start_idx + 65].unsqueeze(0)
        
        # Forward & Backward
        logits, loss = model(x, targets=y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f"   Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
    return model, tokenizer

# 실행
print("\n" + "="*50)
print("🎲 텍스트 생성 테스트")
print("="*50)

model, tokenizer = simple_training_example()

# 생성
generated = generate(model, tokenizer, "The cat", max_tokens=20, temperature=0.8)
print(f"\n📝 생성 결과: '{generated}'")

## 6. Decision Transformer와의 연결

### 6.1 GPT-2 → Decision Transformer 변환

```mermaid
flowchart TB
    subgraph GPT["GPT-2 (언어 모델링)"]
        direction TB
        GIn["Token IDs<br/>(B, T)"]
        GEmb["Token Embedding<br/>(vocab → n_embd)"]
        GTrans["Transformer<br/>Causal Attention"]
        GHead["Vocab Head<br/>(n_embd → vocab)"]
        GOut["Next Token"]
    end

    subgraph DT["Decision Transformer (RL)"]
        direction TB
        DIn["(R, s, a) tuples<br/>(B, T, 3, dim)"]
        DEmb["3개 Embedding<br/>RTG, State, Action"]
        DTrans["Transformer<br/>Causal Attention"]
        DHead["Action Head<br/>(n_embd → act_dim)"]
        DOut["Next Action"]
    end

    GIn --> GEmb --> GTrans --> GHead --> GOut
    DIn --> DEmb --> DTrans --> DHead --> DOut

    GEmb -.->|"변환"| DEmb
    GHead -.->|"변환"| DHead

    style GPT fill:#e3f2fd
    style DT fill:#c8e6c9
    style GTrans fill:#bbdefb
    style DTrans fill:#a5d6a7
```

### 6.2 주요 차이점 비교

```mermaid
flowchart LR
    subgraph Aspect["측면"]
        A1["입력"]
        A2["임베딩"]
        A3["위치 정보"]
        A4["출력"]
        A5["손실"]
    end

    subgraph GPT2["GPT-2"]
        G1["토큰 IDs"]
        G2["Token Embedding"]
        G3["Positional Embedding"]
        G4["다음 토큰 분포"]
        G5["Cross Entropy"]
    end

    subgraph DT["Decision Transformer"]
        D1["(R, s, a) 튜플"]
        D2["3개 Embedding"]
        D3["Timestep Embedding"]
        D4["다음 Action (회귀/분류)"]
        D5["MSE / Cross Entropy"]
    end

    A1 --> G1
    A1 --> D1
    A2 --> G2
    A2 --> D2
    A3 --> G3
    A3 --> D3
    A4 --> G4
    A4 --> D4
    A5 --> G5
    A5 --> D5

    style GPT2 fill:#e3f2fd
    style DT fill:#c8e6c9
```

### 6.3 DT 시퀀스 구성 상세

```mermaid
flowchart TB
    subgraph Input["원본 데이터"]
        RTG["RTG<br/>(B, T, 1)"]
        State["States<br/>(B, T, state_dim)"]
        Action["Actions<br/>(B, T, act_dim)"]
    end

    subgraph Embed["임베딩"]
        RE["RTG Linear<br/>→ (B, T, hidden)"]
        SE["State Linear<br/>→ (B, T, hidden)"]
        AE["Action Linear<br/>→ (B, T, hidden)"]
    end

    subgraph Timestep["Timestep 추가"]
        TE["Timestep Embedding<br/>→ (B, T, hidden)"]
        Add1["RTG + Timestep"]
        Add2["State + Timestep"]
        Add3["Action + Timestep"]
    end

    subgraph Stack["시퀀스 스택"]
        Stack["stack([R, s, a], dim=1)<br/>→ (B, 3, T, hidden)"]
    end

    subgraph Reshape["재배치"]
        Reshape["permute(0,2,1,3)<br/>reshape(B, 3*T, hidden)"]
    end

    subgraph Output["최종 시퀀스"]
        Seq["[R₀, s₀, a₀, R₁, s₁, a₁, ...]<br/>shape: (B, 3*T, hidden)"]
    end

    RTG --> RE --> Add1
    State --> SE --> Add2
    Action --> AE --> Add3
    TE --> Add1
    TE --> Add2
    TE --> Add3
    Add1 --> Stack
    Add2 --> Stack
    Add3 --> Stack --> Reshape --> Seq

    style Seq fill:#c8e6c9
    style Reshape fill:#fff9c4
```

### 6.4 추론 시 RTG 업데이트

```mermaid
sequenceDiagram
    participant User as 🎯 사용자
    participant Model as 🧠 DT Model
    participant Env as 🌍 환경
    participant RTG as RTG 변수

    Note over User,RTG: 추론 시작
    User->>Model: target_return = 100
    User->>RTG: rtg = 100

    Model->>Env: action₀ = model(state, rtg=100)
    Env-->>Model: reward = 10
    Note over Model: rtg = 100 - 10 = 90

    Model->>Env: action₁ = model(state, rtg=90)
    Env-->>Model: reward = 20
    Note over Model: rtg = 90 - 20 = 70

    Model->>Env: action₂ = model(state, rtg=70)
    Note over Model: ...계속...

    Note over Env: rtg ≈ 0 → 목표 달성!
```

In [ ]:
# ============================================================
# 🔄 Decision Transformer 스타일 시퀀스 구성
# ============================================================

def create_dt_sequence(batch_size, seq_len, state_dim, act_dim, hidden_dim):
    """
    Decision Transformer용 시퀀스 구성 예시
    
    GPT-2: [token_0, token_1, token_2, ...]
    DT:     [R_0, s_0, a_0, R_1, s_1, a_1, ...]
    """
    # 가짜 데이터
    rtg = torch.randn(batch_size, seq_len, 1)
    states = torch.randn(batch_size, seq_len, state_dim)
    actions = torch.randn(batch_size, seq_len, act_dim)
    
    # 임베딩 레이어
    rtg_linear = nn.Linear(1, hidden_dim)
    state_linear = nn.Linear(state_dim, hidden_dim)
    action_linear = nn.Linear(act_dim, hidden_dim)
    
    # 임베딩
    rtg_emb = rtg_linear(rtg)       # (B, T, hidden)
    state_emb = state_linear(states)
    action_emb = action_linear(actions)
    
    # 시퀀스 스택: (B, 3, T, hidden)
    stacked = torch.stack([rtg_emb, state_emb, action_emb], dim=1)
    
    # 재배치: (B, 3*T, hidden)
    sequence = stacked.permute(0, 2, 1, 3).reshape(batch_size, 3 * seq_len, hidden_dim)
    
    print(f"📊 DT 시퀀스 구성:")
    print(f"   RTG shape: {rtg.shape} -> 임베딩: {rtg_emb.shape}")
    print(f"   State shape: {states.shape} -> 임베딩: {state_emb.shape}")
    print(f"   Action shape: {actions.shape} -> 임베딩: {action_emb.shape}")
    print(f"   최종 시퀀스: {sequence.shape}")
    print(f"\n   시퀀스 구조: [R_0, s_0, a_0, R_1, s_1, a_1, ...]")
    print(f"   State 위치(1::3)에서 Action 예측!")
    
    return sequence

# 실행
seq = create_dt_sequence(batch_size=2, seq_len=5, state_dim=10, act_dim=4, hidden_dim=32)

## 요약

### ✅ 학습한 내용

1. **Self-Attention**: 각 토큰이 문맳을 기반으로 다른 토큰에 attend
2. **Causal Masking**: 미래 정보 누설 방지
3. **Transformer Block**: Attention + MLP + Residual
4. **GPT-2**: Token/Positional Embedding + Blocks + Head
5. **DT 연결**: (R, s, a) 시퀀스로 RL 재구성

### 📚 다음 단계

- [ ] Decision Transformer 실습 (`phase3_gym_practice.ipynb`)
- [ ] Atari 환경 실습 (`phase4_atari_practice.ipynb`)
- [ ] 하이퍼파라미터 실험

---

### 참고 자료

- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- [Language Models are Unsupervised Multitask Learners (GPT-2)](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)
- [annotated transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)